In [1]:
print("hello world")

hello world


In [5]:
from youtube_transcript_api import YouTubeTranscriptApi,TranscriptsDisabled
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
embedding_model=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

llm=HuggingFaceEndpoint(
    repo_id="katanemo/Arch-Router-1.5B",
    task="text-generation"
)
chat_model=ChatHuggingFace(llm=llm)

### indexing (document ingeions
)

In [75]:
video_id = "poQMJmE57N4"
try:
    fetched_transcript = YouTubeTranscriptApi().fetch(video_id, languages=['en'])
    transcript_list = fetched_transcript.to_raw_data()
    for a in transcript_list:
        print(a)
    transcript = " ".join(chunk["text"] for chunk in transcript_list)
    print(transcript)
    
except TranscriptsDisabled:
    print("No captions available for this video.")
except Exception as e:
    print(f"An error occurred: {e}")


{'text': 'Success is hard until you build systems', 'start': 0.08, 'duration': 4.0}
{'text': "like this. You see, systems don't care", 'start': 2.159, 'duration': 4.401}
{'text': 'how you feel. Systems work on good days', 'start': 4.08, 'duration': 4.08}
{'text': 'and bad days. You see, instead of', 'start': 6.56, 'duration': 3.6}
{'text': 'relying on motivation or even', 'start': 8.16, 'duration': 4.72}
{'text': "discipline, systems create success. It's", 'start': 10.16, 'duration': 4.32}
{'text': "not about trying harder. It's not about", 'start': 12.88, 'duration': 3.04}
{'text': 'waiting to feel ready. It is about', 'start': 14.48, 'duration': 3.28}
{'text': 'building something that sustains you.', 'start': 15.92, 'duration': 3.279}
{'text': 'Something that forces movement,', 'start': 17.76, 'duration': 3.04}
{'text': 'something that makes the right action', 'start': 19.199, 'duration': 3.281}
{'text': 'automatic, and something that makes the', 'start': 20.8, 'duration': 3.6}
{'tex

In [76]:
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks=splitter.create_documents([transcript])
len(chunks)

13

In [77]:
vector_store=Chroma(
    embedding_function=embedding_model,
    collection_name="sample"
)

In [78]:
vector_store.add_documents(chunks)

['8753698b-6c91-4c79-b980-bd358a452f37',
 '754fe270-4e46-4dfc-8137-ee8e30af628d',
 '15dd2527-2dfe-4647-be4e-21e17a0c65ba',
 '32e3cce2-81f7-4436-8473-583e9a861ee2',
 'cacc9288-efe6-4d2a-a344-ee603443ec83',
 'c044d51d-bdf2-4943-b4c2-a874ffa038a4',
 '5600d9ff-42b0-4be8-b2a6-5435fd1461ba',
 '4a17d8a0-e1d9-4d1a-80b2-ba9fe4e3e68e',
 'e4e49a4e-6fc6-43e5-80a2-2b6835707a92',
 '2b380a4e-81ca-476f-953a-43e8a7cd2a1c',
 'bbd1eb2f-163b-4dd2-934b-90d5185cdf7c',
 'c02e6eed-85a8-411d-9bad-6c8ff0cd9436',
 '3a96ac78-c38d-4da1-8585-de52bc7c4033']

In [79]:
vector_store.get(include=["embeddings"])

{'ids': ['eaff2f80-a570-4cc1-b20c-45007bd1de5e',
  '309978e7-763b-4e67-88cb-ae4fc711efc0',
  'a477efd4-a82f-437b-9bf0-d2854e851846',
  'b8ee8a73-4757-4dde-96d3-82649d6e27c1',
  '14f4c660-2fbb-44ec-920e-acfdc1e27a95',
  'a0078897-11ae-4a99-b969-bb3c7289bf1c',
  '0eac19a1-99d0-4ae1-94fa-23b7a69e435d',
  '6ac09df0-a996-4c3f-8f83-8a14104d9931',
  '1d621476-c09c-468b-89b5-5309a9763678',
  'a1eb5a80-57b3-4ab7-8777-7ba256c0f525',
  'ac580a4e-cd2b-4ace-90b2-5d2902bfecb3',
  'bd2cd533-07c9-45d7-94ff-f98fb6946c6d',
  'f8111bcf-1531-4885-a631-2e9eb166fa96',
  '0403033c-0be9-45b2-a749-7776b6be79bb',
  'aca891e7-9a6f-4330-85ef-796a3d62919a',
  'afeca820-db64-4ccc-a687-10933c427c45',
  '358dde25-a2c7-4125-863b-73dc529abb02',
  'f3264c1b-550e-436b-a396-2ae9f1c1275e',
  '6a73323b-079f-4380-973c-92b5e97d4c97',
  'f44cee96-4250-4939-8732-c3e40ae5766d',
  'bceea18c-b26a-44a7-b209-b1c8ce3fc666',
  '0bc33f57-790e-49c6-a57a-b017fbca10ae',
  '33e6b003-d133-4f48-b54a-0a8f282af3c8',
  '9f0dbe31-d09b-4dcf-8859-

In [80]:
retriever=vector_store.as_retriever(search_type="similarity",search_kwargs={"k":4})

In [81]:
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x318a19810>, search_kwargs={'k': 4})

In [70]:
retriever.invoke("how to do detox")

[Document(id='78e82f5e-1478-4e33-b8d5-11b932d274dc', metadata={}, page_content="on these apps, the more you become addicted, the more you want to do it. And it's hard to really snap yourself out when you're already spending a lot of time and you already have this habit that is built in where you just open your phone unnecessarily. You check your phone way too much throughout the day. And the way you really break this cycle is by going on a 24-hour detox. This is like a dopamine detox. You distance yourself from your phone throughout the day. It's always going to be easier to optimize your life if you optimize your environment first because your environment creates your habits and then your habits create your reality. Simply just put your phone somewhere else. Leave it on airplane mode. Leave it in another room. Leave it downstairs. Don't sleep with it in your room. Keep it very far from you. You don't want to just want to detach yourself from it. And by doing that, you're making it tha

In [82]:
prompt=PromptTemplate(
    template="""
        you are a helpful assistant,
        answer only from the provided transcript context.
        if context is unsufficient just say don't know

        {context}
        question: {question}?
        """,
        input_variables=["context","question"]
)

In [89]:
question="what is environment modeling"
retrived_doc=retriever.invoke(question)
retrived_doc

[Document(id='cacc9288-efe6-4d2a-a344-ee603443ec83', metadata={}, page_content="but you can't have a working system unless you yourself are that person. Unless your beliefs are aligned with where you want to go, with the life you want to live, the discipline, the consistency, and the focus. You see, identity is such an important part of achievement, excellence, and also just discipline. You see, I created something called the discipline protocol, which is just a free guide that you can download in the link below. Basically, it consists of the systems I use for discipline. Check that out and let's move on to part number two, environmental modeling. Let's take a look at the two environment rule. You see, mixing energies and environments is a recipe for disaster. You see, the brain associates energy and location faster than habits. So, you need to segment your environments by having a space that is just exclusively for work, another space that is exclusively for entertainment and relaxing

In [107]:
final_prompt=prompt.invoke({"context":retrived_doc,"question":question})

In [108]:
answer=chat_model.invoke(final_prompt)
print(answer.content)

Environment modeling involves organizing your living spaces into distinct areas dedicated to work, relaxation, and entertainment, ensuring that each area remains isolated from others. This practice helps maintain a clear separation between productive activities and leisure time, promoting better concentration and reducing distractions.


In [93]:
from langchain_core.runnables import RunnableParallel,RunnablePassthrough,RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [94]:
def formet_docs(ret_docs):
    return "\n\n".join(doc.page_content for doc in ret_docs)

In [95]:
pll_chain=RunnableParallel({
    "context": retriever | RunnableLambda(formet_docs),
    "question":RunnablePassthrough()
})

In [97]:
parser=StrOutputParser()

In [99]:
pll_chain.invoke("enviroment modeling")


{'context': 'but you can\'t have a working system unless you yourself are that person. Unless your beliefs are aligned with where you want to go, with the life you want to live, the discipline, the consistency, and the focus. You see, identity is such an important part of achievement, excellence, and also just discipline. You see, I created something called the discipline protocol, which is just a free guide that you can download in the link below. Basically, it consists of the systems I use for discipline. Check that out and let\'s move on to part number two, environmental modeling. Let\'s take a look at the two environment rule. You see, mixing energies and environments is a recipe for disaster. You see, the brain associates energy and location faster than habits. So, you need to segment your environments by having a space that is just exclusively for work, another space that is exclusively for entertainment and relaxing and doing that reset. And you see a lot of us do everything in 

In [109]:
final_chain= pll_chain | prompt| chat_model|parser


In [110]:
final_chain.invoke("can you summorise the video ")

'The video provides a step-by-step guide called "Five Steps Framework" to help viewers build better habits and become more disciplined. It emphasizes focusing on the core issues (media overload) and using this knowledge to achieve consistency in daily routine.'